In [1]:
import math
import itertools

import pandas as pd
from sqlalchemy.sql import select

from predict_aqi.load_data import (
    AirLocation, get_db_session, load_air_location_data, load_measurement_data,
    load_nearby_location_measurement_data
)

# Select some airlocation with bad aqi
session = get_db_session()
s = select([AirLocation]).where(AirLocation.aqi > 100)
some_airlocation_id = session.execute(s).scalar()
some_airlocation = load_air_location_data(some_airlocation_id)

print("{}, {}".format(some_airlocation.short_name, some_airlocation.en_country_name))

measurements = load_measurement_data(some_airlocation_id)

/opt/conda/lib/python3.5/site-packages/sqlalchemy/dialects/postgresql/base.py:2503: SAWarning: Did not recognize type 'geometry' of column 'coordinates'
  (attype, name))


Mudanjiang, China


In [9]:
from predict_aqi.model import generate_AQI_inputs_and_outputs
from predict_aqi import config

indices_ahead_to_predict = range(2, config.NUMBER_AHEAD_TO_PREDICT + 2, 2)
indices_behind_to_use = range(1, 25)
df, input_columns, output_columns = generate_AQI_inputs_and_outputs(
    measurements, indices_behind_to_use, indices_ahead_to_predict
)

ImportError: cannot import name 'generate_AQI_inputs_and_outputs'

In [8]:
# Define functions for generate_predictions method

def first_step_format_inputs_outputs(all_data):
    return all_data, input_columns, output_columns

 
def first_step_split_function(all_data, input_columns, output_columns):
    # Split by the first 90% of data. That means we're our testing set is extrapolation.
    row_count = all_data.count()[0]
    split_row = round(row_count * 0.90)
    # x_train, y_train, x_test, y_test
    return (all_data[input_columns][:row_count], 
            all_data[output_columns][:row_count],
            all_data[input_columns][row_count:],
            all_data[output_columns][row_count:])


def second_step_format_inputs_outputs(all_data):
    # do time input stuff to all_data
    # return the right columns
    return all_data


second_step_split_function = first_step_split_function

37593


In [5]:
# Select classifier to use
from sklearn.neural_network import MLPRegressor

first_step_regressor = MLPRegressor()
second_step_regressor = MLPRegressor()

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46]


In [11]:
from predict_aqi.model import generate_predictions
df = generate_predictions(
    df,
    first_step_format_inputs_outputs,
    first_step_split_function,
    second_step_format_inputs_outputs,
    second_step_split_function,
    first_step_regressor,
    second_step_regressor,
    indices_ahead_to_predict,
)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features[feature_columns])
    end = time.time()
    print("Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return y_pred

print("Making training predictions...")
y_train_pred = predict_labels(clf, x_train, y_train)
print("Making test predictions...")
y_test_pred = predict_labels(clf, x_test, y_test)

Making training predictions...
Predicting labels using MLPRegressor...
Done!
Prediction time (secs): 0.032
Making test predictions...
Predicting labels using MLPRegressor...
Done!
Prediction time (secs): 0.005


In [47]:
from sklearn.metrics import mean_absolute_error

# Score predictions
print("Mean absolute error for predicting training set (interpolation): {}".format(mean_absolute_error(y_train, y_train_pred) * MAX_AQI))
print("Mean absolute error for predicting test set (interpolation): {}".format(mean_absolute_error(y_test, y_test_pred) * MAX_AQI))
print("Mean squared error for 0-ahead prediction test set: {}".format(
    mean_absolute_error(y_test['0_ahead_normalized_aqi'], [r[0] for r in y_test_pred]) * MAX_AQI
))
print("Mean squared error for 11-ahead prediction test set: {}".format(
    mean_absolute_error(y_test['11_ahead_normalized_aqi'], [r[11] for r in y_test_pred]) * MAX_AQI
))
print("Mean squared error for 23-ahead prediction test set: {}".format(
    mean_absolute_error(y_test['23_ahead_normalized_aqi'], [r[23] for r in y_test_pred]) * MAX_AQI
))
print("Mean squared error for 35-ahead prediction test set: {}".format(
    mean_absolute_error(y_test['35_ahead_normalized_aqi'], [r[35] for r in y_test_pred]) * MAX_AQI
))
print("Mean squared error for 47-ahead prediction test set: {}".format(
    mean_absolute_error(y_test['47_ahead_normalized_aqi'], [r[47] for r in y_test_pred]) * MAX_AQI
))

Mean absolute error for predicting training set (interpolation): 19.230963908951104
Mean absolute error for predicting test set (interpolation): 19.267614587656553
Mean squared error for 0-ahead prediction test set: 5.2740734909409275
Mean squared error for 11-ahead prediction test set: 16.79834040349421
Mean squared error for 23-ahead prediction test set: 20.59979318925587
Mean squared error for 35-ahead prediction test set: 22.811270845452732
Mean squared error for 47-ahead prediction test set: 24.077800385773706
